## Pandas, SQL, and the Grammar of Data

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns
sns.set_style('whitegrid')

In [2]:
from sqlite3 import dbapi2 as sq3
import os

### Create schema

In [3]:
ourschema = '''
DROP TABLE IF EXISTS 'candidates';
DROP TABLE IF EXISTS 'contributors';
CREATE TABLE 'candidates' (
'id' INTEGER PRIMARY KEY NOT NULL,
'first_name' VARCHAR,
'last_name' VARCHAR,
'middle_name' VARCHAR,
'party' VARCHAR NOT NULL
);

CREATE TABLE 'contributors' (
'id' INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
'last_name' VARCHAR,
'first_name' VARCHAR,
'middle_name' VARCHAR,
'street_1' VARCHAR,
'street_2' VARCHAR,
'city' VARCHAR,
'state' VARCHAR,
'zip' VARCHAR,
'amount' INTEGER,
'date' DATETIME,
'candidate_id' INTEGER NOT NULL,
FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
'''

In [4]:
PATHSTART = '.'
def get_db(dbfile):
    
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db



In [5]:
def init_db(dbfile, schema):
    
    '''Create the database tables'''
    
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [6]:
dfcand=pd.read_csv("https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture4/candidates.txt", sep='|')
dfcand

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D
5,22,Rudolph,Giuliani,NaN,R
6,24,Mike,Gravel,NaN,D
7,16,Mike,Huckabee,NaN,R
8,30,Duncan,Hunter,NaN,R
9,31,Dennis,Kucinich,NaN,D


In [7]:
dfcwci=pd.read_csv("https://raw.githubusercontent.com/cs109/2015/master/Lectures/Lecture4/contributors_with_candidate_id.txt", sep='|')
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [8]:
del dfcwci['id']
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Initializing the database


In [9]:
db = init_db('cancont.db', ourschema)

Populating with Pandas!!

In [10]:
dfcand.to_sql('candidates', db, if_exists='append', index= False)
dfcwci.to_sql('contributors', db, if_exists='append', index= False)

In [11]:
dfcwci.shape

(175, 11)

In [12]:
sel="""
SELECT * FROM candidates;
"""
c=db.cursor().execute(sel)

In [13]:
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

Making Queries

In [14]:
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [15]:
make_query("SELECT * FROM candidates")

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

Data manipulation in Pandas and SQL.

In [16]:
dfcwci.head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


Selecting State Virginia and Amount less than \$400

In [17]:
dfcwci.query("state == 'VA' & amount < 400")

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [18]:
dfcwci[(dfcwci['state'] == 'VA') & (dfcwci['amount'] < 400)]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [19]:
my_query = make_query("SELECT * FROM contributors WHERE state = 'VA' AND amount < 400")

In [20]:
pd.DataFrame(my_query, 
             columns=[e[1] for e in make_query("PRAGMA table_info(contributors)")])

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [21]:
# Make a function to output a dataframe
def make_frame(my_data, table):
    
    
    
    col_names = columns=[e[1] for e in make_query("PRAGMA table_info("+str(table)+")")]
    
    return pd.DataFrame(my_data, 
             columns=col_names)
    

#### QUERY

In [22]:
out = make_query("SELECT * FROM contributors WHERE state is NULL")
make_frame(out,'contributors')

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,126,BOURNE,TRAVIS,None,LAGE KAART 77,None,BRASSCHATT,None,2930,-500,2008-11-20,35


In [23]:
dfcwci[dfcwci['state'].isnull()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
125,BOURNE,TRAVIS,NaN,LAGE KAART 77,NaN,BRASSCHATT,NaN,2930,-500.0,2008-11-20,35


In [24]:
out = make_query("SELECT * FROM contributors WHERE state IS NOT NULL")
make_frame(out, 'contributors').shape

(174, 12)

In [25]:
dfcwci[dfcwci['state'].notnull()].shape

(174, 11)

In [26]:
out=make_query("SELECT * FROM contributors WHERE state IN ('VA','WA');")
make_frame(out,'contributors').head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,1,Agee,Steven,None,549 Laurel Branch Road,None,Floyd,VA,24091,500.00,2007-06-30,16
1,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
2,63,BURKE,SUZANNE,M.,3401 EVANSTON,None,SEATTLE,WA,981038677,-700.00,2008-03-05,22
3,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
4,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
5,101,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.00,2008-02-08,34
6,107,Aaronson,Rebecca,None,2000 Village Green Dr Apt 12,None,Mill Creek,WA,980125787,100.00,2008-02-14,34
7,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [27]:
dfcwci[dfcwci['state'].isin(['VA','WA'])].head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.00,2007-06-30,16
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.00,2007-09-19,20
62,BURKE,SUZANNE,M.,3401 EVANSTON,NaN,SEATTLE,WA,981038677,-700.00,2008-03-05,22
77,Ranganath,Anoop,NaN,2507 Willard Drive,NaN,Charlottesville,VA,22903,-100.00,2008-04-21,32
88,Perreault,Louise,NaN,503 Brockridge Hunt Drive,NaN,Hampton,VA,23666,-34.08,2008-04-21,32
100,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-08,34
106,Aaronson,Rebecca,NaN,2000 Village Green Dr Apt 12,NaN,Mill Creek,WA,980125787,100.00,2008-02-14,34
145,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,NaN,FAIRFAX,VA,220308440,50.00,2007-09-30,35


In [28]:
out=make_query("SELECT * FROM contributors WHERE amount BETWEEN 10 AND 50;")
make_frame(out,'contributors').head(10)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,3,Ahrens,Don,None,4034 Rennellwood Way,None,Pleasanton,CA,94566,50,2007-06-18,16
1,19,Ardle,William,None,412 Dakota Avenue,None,Springfield,OH,45504,50,2007-06-28,16
2,26,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,50,2007-07-30,20
3,27,Buckler,Steve,None,24351 Armada Dr,None,Dana Point,CA,926291306,25,2007-08-16,20
4,35,Buck,Barbara,None,1780 NE 138th St,None,North Miami,FL,331811316,50,2007-09-13,20
5,36,Buck,Barbara,None,1780 NE 138th St,None,North Miami,FL,331811316,50,2007-07-19,20
6,39,Buchanek,Elizabeth,None,7917 Kentbury Dr,None,Bethesda,MD,208144615,50,2007-09-30,20
7,50,Harrison,Ryan,None,2247 3rd St,None,La Verne,CA,917504918,25,2007-07-26,20
8,102,Aarons,Elaine,None,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25,2008-02-26,34
9,105,Aaron,Shirley,None,101 Cherry Ave,None,Havana,FL,323331311,50,2008-02-29,34


In [29]:
dfcwci.query("10 <= amount <= 50").head(10)

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
18,Ardle,William,NaN,412 Dakota Avenue,NaN,Springfield,OH,45504,50.0,2007-06-28,16
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
34,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-09-13,20
35,Buck,Barbara,NaN,1780 NE 138th St,NaN,North Miami,FL,331811316,50.0,2007-07-19,20
38,Buchanek,Elizabeth,NaN,7917 Kentbury Dr,NaN,Bethesda,MD,208144615,50.0,2007-09-30,20
49,Harrison,Ryan,NaN,2247 3rd St,NaN,La Verne,CA,917504918,25.0,2007-07-26,20
101,Aarons,Elaine,NaN,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,26733300,25.0,2008-02-26,34
104,Aaron,Shirley,NaN,101 Cherry Ave,NaN,Havana,FL,323331311,50.0,2008-02-29,34


#### SORT

In [30]:
dfcwci.sort_values('amount').head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
90,Kazor,Christopher,M,707 Spindletree ave,NaN,Naperville,IL,60565,-2592.0,2008-04-21,32
72,BRUNO,JOHN,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
64,BURKE,DONALD,J.,12 LOMPOC,NaN,RANCHO SANTA MARGA,CA,926881817,-2300.0,2008-03-11,22
73,BRUNO,IRENE,NaN,10136 WINDERMERE CHASE BLVD.,NaN,GOTHA,FL,347344707,-2300.0,2008-03-06,22
74,BROWN,TIMOTHY,J.,26826 MARLOWE COURT,NaN,STEVENSON RANCH,CA,913811020,-2300.0,2008-03-06,22


In [31]:
dfcwci.sort_values('amount',ascending=False).head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20
159,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,NaN,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
15,Anthony,John,NaN,211 Long Island Drive,NaN,Hot Springs,AR,71913,2300.0,2007-06-12,16
33,Buck,Blaine,M,45 Eaton Ave,NaN,Camden,ME,48431752,2300.0,2007-09-30,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20


In [32]:
out=make_query("SELECT * FROM contributors ORDER BY amount;")
make_frame(out,'contributors').head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,91,Kazor,Christopher,M,707 Spindletree ave,None,Naperville,IL,60565,-2592.0,2008-04-21,32
1,30,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,-2300.0,2007-08-14,20
2,52,BYINGTON,MARGARET,E.,2633 MIDDLEBORO LANE N.E.,None,GRAND RAPIDS,MI,495061254,-2300.0,2008-03-03,22
3,53,BYERS,BOB,A.,13170 TELFAIR AVENUE,None,SYLMAR,CA,913423573,-2300.0,2008-03-07,22
4,55,BUSH,KRYSTIE,None,P.O. BOX 61046,None,DENVER,CO,802061046,-2300.0,2008-03-06,22


In [33]:
out=make_query("SELECT * FROM contributors ORDER BY amount DESC;")
make_frame(out,'contributors').head()


,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,31,Buckel,Linda,None,PO Box 683130,None,Park City,UT,840683130,4600.0,2007-08-14,20
1,160,ABATE,MARIA,ELENA,1291 NIGHTINGALE AVENUE,None,MIAMI SPRINGS,FL,331663832,2600.0,2008-01-25,37
2,14,Altes,R.D.,None,8600 Moody Road,None,Fort Smith,AR,72903,2300.0,2007-06-21,16
3,16,Anthony,John,None,211 Long Island Drive,None,Hot Springs,AR,71913,2300.0,2007-06-12,16
4,22,Baker,David,None,2550 Adamsbrooke Drive,None,Conway,AR,72034,2300.0,2007-04-11,16


#### SELECT-COLUMNS

In [34]:
dfcwci[['first_name', 'amount']].head()

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0


In [35]:
out = make_query("SELECT first_name, amount FROM contributors;")
pd.DataFrame(out, columns=['first_name', 'amount']).head()

,first_name,amount
0,Steven,500.0
1,Don,250.0
2,Don,50.0
3,Don,100.0
4,Charles,100.0


#### SELECT-DISTINCT

In [36]:
dfcwci[['last_name','first_name']].count()

last_name     175
first_name    175
dtype: int64

In [37]:
dfcwci[['last_name','first_name']].drop_duplicates().head()

,last_name,first_name
0,Agee,Steven
1,Ahrens,Don
4,Akin,Charles
5,Akin,Mike
6,Akin,Rebecca


In [38]:
out=make_query("SELECT DISTINCT last_name, first_name FROM contributors;")
pd.DataFrame(out, columns=['first_name', 'last_name']).head()

,first_name,last_name
0,Agee,Steven
1,Ahrens,Don
2,Akin,Charles
3,Akin,Mike
4,Akin,Rebecca


#### AGGREGATE

In [39]:
dfcwci.describe()

,zip,amount,candidate_id
count,1.750000e+02,175.000000,175.000000
mean,3.780014e+08,3.418114,28.000000
std,3.628278e+08,1028.418999,7.823484
min,2.474000e+03,-2592.000000,16.000000
25%,9.336700e+04,-175.000000,20.000000
50%,3.233313e+08,100.000000,32.000000
75%,7.816946e+08,300.000000,35.000000
max,9.951532e+08,4600.000000,37.000000


In [40]:
dfcwci.max()

last_name             Waddell
first_name             ZAINUL
street_1        PO Box 683130
city                  Yardley
zip                 995153207
amount                   4600
date               2008-11-20
candidate_id               37
dtype: object

In [41]:
# Who gave the most amount.
dfcwci[dfcwci['amount'] == dfcwci['amount'].max()]

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
30,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,4600.0,2007-08-14,20


In [42]:
out=make_query("SELECT *, MAX(amount) FROM contributors;")
out
# make_frame(out, cont_cols+['maxamt'])

[(31,
  'Buckel',
  'Linda',
  None,
  'PO Box 683130',
  None,
  'Park City',
  'UT',
  '840683130',
  4600,
  '2007-08-14',
  20,
  4600)]

#### JOINS

Contributions to Obama

In [43]:
obamaid = dfcand[dfcand['last_name'] == 'Obama']['id'].values[0]
obamaid

20

In [44]:
dfcwci[dfcwci['candidate_id']==obamaid].head()

,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
25,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,50.0,2007-07-30,20
26,Buckler,Steve,NaN,24351 Armada Dr,NaN,Dana Point,CA,926291306,25.0,2007-08-16,20
27,Buckheit,Bruce,NaN,8904 KAREN DR,NaN,FAIRFAX,VA,220312731,100.0,2007-09-19,20
28,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,2300.0,2007-08-14,20
29,Buckel,Linda,NaN,PO Box 683130,NaN,Park City,UT,840683130,-2300.0,2007-08-14,20


In [48]:
dfcwci.merge(dfcand, left_on='candidate_id', right_on = 'id').head()

,last_name_x,first_name_x,middle_name_x,street_1,street_2,city,state,zip,amount,date,candidate_id,id,first_name_y,last_name_y,middle_name_y,party
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,16,Mike,Huckabee,NaN,R
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,16,Mike,Huckabee,NaN,R
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,16,Mike,Huckabee,NaN,R
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,16,Mike,Huckabee,NaN,R
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,16,Mike,Huckabee,NaN,R


Outer Join

In [50]:
dfcwci.merge(dfcand, left_on="candidate_id", right_on="id", how="left").head()

,last_name_x,first_name_x,middle_name_x,street_1,street_2,city,state,zip,amount,date,candidate_id,id,first_name_y,last_name_y,middle_name_y,party
0,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16,16,Mike,Huckabee,NaN,R
1,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16,16,Mike,Huckabee,NaN,R
2,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16,16,Mike,Huckabee,NaN,R
3,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16,16,Mike,Huckabee,NaN,R
4,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16,16,Mike,Huckabee,NaN,R


#### Pandas SQL

In [52]:
pd.read_sql('SELECT * FROM candidates where party = "D"', db)

,id,first_name,last_name,middle_name,party
0,20,Barack,Obama,None,D
1,24,Mike,Gravel,None,D
2,26,John,Edwards,None,D
3,29,Bill,Richardson,None,D
4,31,Dennis,Kucinich,None,D
5,33,Joseph,Biden,None,D
6,34,Hillary,Clinton,R.,D
7,39,Christopher,Dodd,J.,D
